In [2]:
import json
import pandas as pd
import glob
import os

In [5]:
for annotation in glob.glob('/home/gujiashe/Tk-Instruct/induction_data/annotations/*.json'):
    with open(annotation, 'r') as task_f:
        s = task_f.read()
        task_data = json.loads(s)
    task_path = annotation.split('/')
    task_path[-2] = 'induction_input'
    task_definition = task_data['annotations'][0]
    task_path = '/'.join(task_path)
    with open(task_path, 'r') as task_f:
        s = task_f.read()
        task_data = json.loads(s)
        task_data['Definition'] = [task_definition]
        
        task_path = task_path.split('/')
        task_path[-2] = 'tasks'
        _task_path = '/'.join(task_path)
        with open(_task_path, 'w') as _task_f:
            json.dump(task_data, _task_f, indent=2)
        # break

In [9]:
f = open('/home/gujiashe/Tk-Instruct/induction_data/tasks/cause_and_effect.json', 'r')
ex1 = json.load(f)


In [23]:
len(list(ex1['examples'])[:10001])

100

In [18]:
for i in ex1['examples']['1']['input'].split('\n\n'):
    if i.startswith('Input:'):
        input_ = i.split('\n')[0].strip('Input: ')
        output_ = i.split('\n')[1].strip('Output: ')
        print(input_)
        print(output_)




Sentence 1: The dinosaurs became extinct. Sentence 2: A large object hit the Earth.
A large object hit the Earth.
Sentence 1: My friend got a promotion. Sentence 2: My friend celebrated with their family.
My friend got a promotion.
Sentence 1: The ship capsized. Sentence 2: The wind blew strong.
The wind blew strong.
Sentence 1: The soda went flat. Sentence 2: The bottle was left open.
The bottle was left open.
Sentence 1: The company's posted strong earnings. Sentence 2: The company's stock went up.
The company's posted strong earnings.


In [21]:
import random
instances = list(ex.values())
random.shuffle(instances)

In [9]:
len(glob.glob("/home/gujiashe/Tk-Instruct/data/tasks/*.json"))
# 😧🙈🙂😘😡😩🤬😂🤷 🏻🙄😔😮👊🇺🇸👍😈😎💪🤣😀😍😬 😊💃
# 

1613

In [8]:
tasks = {}


tasks['Categories'] = []
tasks['name'] = []
for i in range(1, 11):
    tasks['Example_' + str(i) + '_input'] = []
    tasks['Example_' + str(i) + '_output'] = []
    tasks['Example_' + str(i) + '_explanation'] = []
paths = []
task_dir = '/home/gujiashe/Tk-Instruct/data/tasks'
path = '/home/gujiashe/Tk-Instruct/data/splits/default/test_tasks.txt'
with open(path, encoding="utf-8") as split_f:
    for line in split_f:
        task_name = line.strip()
        task_path = os.path.join(task_dir, task_name + ".json")
        with open(task_path, "r") as task_f:
            s = task_f.read()
            task = json.loads(s)
            if len(task["Input_language"])>1 or task["Input_language"][0] != "English":
                continue
            if len(task["Output_language"])>1 or task["Output_language"][0] != "English":
                continue

            i = 1
            for example in task['Positive Examples']:
                if i>10:
                    break
                tasks['Example_' + str(i) + '_input'].append(example['input'])
                tasks['Example_' + str(i) + '_output'].append(example['output'])
                tasks['Example_' + str(i) + '_explanation'].append('Positive. ' + example['explanation'])
                i+=1
                


            for example in task['Negative Examples']:
                if i>10:
                    break
                tasks['Example_' + str(i) + '_input'].append(example['input'])
                tasks['Example_' + str(i) + '_output'].append(example['output'])
                tasks['Example_' + str(i) + '_explanation'].append('Negative. ' + example['explanation'])
                i+=1
                
            
            for example in task['Instances']:
                if i>10:
                    break
                tasks['Example_' + str(i) + '_input'].append(example['input'])
                tasks['Example_' + str(i) + '_output'].append(example['output'][0])
                tasks['Example_' + str(i) + '_explanation'].append("")
                i+=1

            Categories = task['Categories'][0]
            name = task_path.split("/")[-1].split(".")[0]
            tasks['name'].append('_'.join(task_name.split('_')[1:]))
            tasks['Categories'].append(Categories)
            paths.append(task_path)
                # break
tasks = pd.DataFrame(tasks)

In [10]:
tasks

,Categories,name,Example_1_input,Example_1_output,Example_1_explanation,Example_2_input,Example_2_output,Example_2_explanation,Example_3_input,Example_3_output,...,Example_7_explanation,Example_8_input,Example_8_output,Example_8_explanation,Example_9_input,Example_9_output,Example_9_explanation,Example_10_input,Example_10_output,Example_10_explanation
0,Title Generation,xlsum_title_generation,"But Eluned Morgan conceded that it would be ""d...",NHS Wales: Court action if trade deals affect ...,Positive. The output phrase is the appropriate...,By Jon Welch and Paul MoseleyBBC News Details ...,University of East Anglia in students' persona...,Positive. University of East Anglia in student...,Media reports say Stanislav Bogdanovich and Al...,Young Ukraine chess couple 'killed by laughing...,...,,By Katherine SellgrenBBC News education report...,Schools 'cut staff due to funding shortages',,By Leo KelionTechnology desk editor It describ...,Intel unveils Project Alloy 'merged reality' h...,,"New Zealand-born Sgt Ratana, known as Matt, wa...",Croydon police officer shot dead named as Sgt ...,
1,Coreference Resolution,gap_fill_the_blank_coreference_resolution,"Dorothy Moore (born 13 October 1946, Jackson, ...",her,Positive. The sentence begins with the name Do...,Groucutt began his musical career at 15 as Rik...,he,Positive. It is mentioned that Groucutt plays ...,After the 1980 elections he became Minister fo...,him,...,,"**Ellen Arden** (Monroe), a photographer and m...",Her,,"Another brother, Dr. Aswin W. Sastrowardoyo, i...",She,,"Also, the twin roles of The Corsican Brothers ...",he,
2,Textual Entailment,esnli_classification,A teenager blows a whistle and wears white fac...,C,Positive. Either a teenager wears face paint o...,Small girl in a pink dress is playing electron...,E,Positive. The girl is wearing a pink dress.,"Five young men are in a loft, with one holding...",N,...,,Two men spar in martial arts gear. <sep> There...,E,,Some youngsters are playing a card game <sep> ...,N,,A crowd of people stand in street in front of ...,C,
3,Textual Entailment,scitail1.1_classification,Premise: Lyme Disease is caused by a bacterium...,entails,Positive. The premise sentence agrees with the...,Premise: Corolla Collective term for all the p...,entails,Positive. The premise sentence explains the me...,Premise: This can be dangerous to both plants ...,neutral,...,Negative. The premise states the name for Mari...,"Premise: Vertebrates are named for vertebrae, ...",neutral,Negative. The usage of the word 'or' between t...,Premise: As threats to bird populations grow a...,neutral,,Premise: The hagfish has no cerebrum or cereb...,entails,
4,Textual Entailment,mnli_contradiction_classification,Statement: Next to the MGM Grand you will find...,3,Positive. It is said in the statement that the...,"Statment: I've forgotten his name now, confess...",3,"Positive. Tuppence had forgotten the name, so ...",Statement: One of the first organizational rea...,1,...,,Statement: EPA responded with a discussion of ...,2,,"Statement: You're not much to look at, but you...",1,,Statement: For the purposes of this discussion...,3,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
114,Question Rewriting,disfl_qa_text_modication,Why was uh where was the Rhine regulated with ...,Where was the Rhine regulated with an upper ca...,Positive. The above disfluent question is corr...,What kind of committee considered legislation ...,What kind of committee considered legislation ...,Positive. The above disfluent question is corr...,When degradation no economic inequality is sma...,"When economic inequality is smaller, more wast...",...,,How long does flagella no sorry Cilia get?,Cilia can g ow up too what length?,,How do socialists think the financial assets n...,How do socialists think the means of productio...,,Where did umm no I mean when did the Holocene ...,When did the Holocene end?,
115,Keyword Tagging,ohsumed_yes_no_answer_generat

In [11]:
tasks.to_csv("tasks.csv", index=None)

In [12]:
with open('tasks.csv', 'r', encoding='utf-8') as f:
    x = f.read()
    with open('tasks1.csv', 'w', encoding='utf-8') as ff:
        ff.write(''.join(char for char in x if ord(char) < 0x10000 and not repr(char).startswith("'\\x")))

In [15]:
import pandas as pd
tasks = pd.read_csv("tasks1.csv")
tasks.iloc[:3].to_csv("tasks_subset.csv", index=None)